# LLMeter with Amazon Bedrock

This notebook demonstrates how you can use LLMeter to measure latency of LLMs hosted on [Amazon Bedrock](https://aws.amazon.com/bedrock/), for comparing between available models or understanding the impact of your workload's prompt (input) and completion (output) lengths on response time.

We start with the basics of invoking LLMs through LLMeter and running request batches - then discuss the higher-level *experiment* offered by LLMeter for length-based latency heatmapping.

## Setting Up

First, ensure your environment has LLMeter installed (and its optional `plotting` utilities) ready to run the example.

In [ ]:
%pip install "llmeter[plotting]<1"

In [ ]:
# External Dependencies:
from llmeter.endpoints import BedrockConverse, BedrockConverseStream
from llmeter.runner import Runner
from upath import UPath  # Combined APIs for accessing cloud or local storage

This notebook assumes you're running in an environment with [configured AWS API credentials](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html) (e.g. through environment variables, profiles, or similar), and that the configured identity has [permissions](https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html) to `bedrock:InvokeModel` and `bedrock:InvokeModelWithResponseStream`. You'll also need to [enable access](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) to your chosen foundation model(s), as this is not granted by default.

LLMeter's `BedrockConverse` and `BedrockConverseStream` endpoints for Amazon Bedrock use the [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html).

Refer to the [model IDs table](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html) and [list of models that support the Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference-supported-models-features.html) this API, to configure an ID below for a supported model you'd like to evaluate:


In [ ]:
model_id = None  # <-- Choose a Bedrock model ID from the above-linked list that supports Converse API

if not model_id:
    raise ValueError("Please set a valid model ID!")

For testing purposes, we can generate a sample payload matching the [Bedrock Converse API format](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html) using a convenience method of the LLMeter `BedrockConverse` class as shown below.

For the full Converse API specification, you can check the [documentation here](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html).

In [ ]:
sample_payload = BedrockConverse.create_payload(
    "Create a list of 3 pop songs.",
    max_tokens=512,
    system=[{"text": "you're an expert in pop and indie music"}],
)
sample_payload

## Basic inference and test runs

To measure the latency and throughput of the model (including the network latency from the client where LLMeter is running to the actual endpoint), we start by creating a [`llmeter.Endpoint`](../llmeter/endpoints/base.py) object specifically for Bedrock deployments.

In [ ]:
# Create an LLMeter Endpoint object for your chosen Bedrock model:
bedrock_endpoint = BedrockConverse(
    model_id=model_id,
    # Optionally, you can target a different AWS Region than your default:
    # region="us-west-2",
)

We can now verify that LLMeter is correctly capturing the output of the endpoint

In [ ]:
response = bedrock_endpoint.invoke(payload=sample_payload)
print(response)

You should see that `time_to_last_token` (the overall response time), `num_tokens_input` and `num_tokens_output` (the actual consumed input and output token counts) are captured in the response. `time_to_first_token` is null in this case because it's only used in streaming APIs (see next section).

Of course a single data point give us much confidence about typical endpoint performance.

We can use the [`Runner`](../llmeter/runner.py) class to set up a basic test run and calculate statistics.
- `clients` configures the number of concurrent (thread-based) clients that will send requests to the endpoint.
- `n_requests` is the total number of consecutive requests **each client** should perform (so the endpoint will receive clients * n_requests requests in total).
- `payload`s are the sample request(s) that should be used for the test. If `n_requests=None`, each client will just iterate through this list. Otherwise, the list will be shuffled to generate the required number of requests.
- `output_path` can be a local path or `s3://...` URI, where the test results should be saved.

The `run()` method is an async function, and to execute it in a Jupyter notebook it requires to use the `await` keyword!

In [ ]:
endpoint_test = Runner(
    bedrock_endpoint,
    output_path=f"outputs/{bedrock_endpoint.model_id}",
)
results = await endpoint_test.run(payload=sample_payload, n_requests=3, clients=3)

Once the test is completed, we can check the overall statistics aggregated from all the clients and requests:

In [ ]:
print(results)

We can also drill down to the individual responses if needed, as shown below.

In [ ]:
results.responses

If the `output_path` was set, we can also find this information saved to files in the provided location:

In [ ]:
print(f"Contents of {results.output_path}\n----")
for sub in UPath(results.output_path).iterdir():
    print(sub)

## Streaming responses and Time-to-First-Token

Many LLM providers support *streaming* APIs which reduce user-perceived latency by sending the response in chunks rather than waiting for the whole generation to finish. In these contexts, the `time_to_first_token` is also a useful metric to understand how long consumers will have to wait before they can start to process the response.

To use Bedrock's streaming API, we can instead connect with an LLMeter `BedrockConverseStream`:

In [ ]:
bedrock_endpoint_stream = BedrockConverseStream(
    model_id=model_id,
    # As before, you could target a different AWS Region than your default:
    # region="us-west-2",
)

As before, we can first test that the `Endpoint` is correctly configured.

In [ ]:
response = bedrock_endpoint_stream.invoke(payload=sample_payload)
print(response)

Once we're satisfied that the `Endpoint` is working as expected and we're now able to capture the `time_to_first_token` metric on the response - we can set up out test runner and `run()` it, as we did for the non-streaming case above.

In [ ]:
endpoint_stream_test = Runner(
    bedrock_endpoint_stream,
    output_path=f"outputs/{bedrock_endpoint_stream.model_id}",
)
results_stream = await endpoint_stream_test.run(
    payload=sample_payload, clients=20, n_requests=5
)

We can view the summary statistics and individual responses as before, but now have a additional statistics to work with:

> ℹ️ **Note:** The `time_to_last_token` may differ between streaming and non-streaming invocations, so it's important to test the invocation method you'll actually be using. Don't choose to test streaming only because it provides additional statistics!

In [ ]:
print(results_stream)

In [ ]:
results_stream.responses

Now you've seen the core mechanics of running tests with LLMeter, let's explore some higher-level *experiments* that use these low-level runners under the hood.

## Mapping latency by input & output token counts

For many LLMs, the time to process a request can significantly depend on the length (in number of tokens) of the input provided and the output generated.

We can produce a heatmap showing how latency varies by these factors, to give an idea of how optimizing your input length or generation lengths might affect the response times observed by users.

The `LatencyHeatmap` experiment automatically generates a set of request payloads with varying (approximate) input lengths and uses it to test the endpoint.

To construct the requests, we need a base text to use as a seed. The semantic aspects are not particularly important, so any sufficiently long text can serve the purpose - but remember that many LLMs have their own internal guardrails, so it's possible that the model might decline to respond in some cases.

We'd like the generated reply to be limited by the `max_tokens` parameter (so the heatmap can measure latency for various output lengths), so will engineer a prompt that encourages the model to generate as long a response as possible from the seed text:

In [ ]:
def prompt_fn(prompt, **kwargs):
    formatted_prompt = f"Create a story based on the following prompt: {prompt}"
    return BedrockConverseStream.create_payload(formatted_prompt, **kwargs)

With a seed text and prompt generation function, we're ready to set up our latency heatmapping experiment.

- The `source_file` and `create_payload_fn` will be used to generate requests with various input lengths.
- The set of `input_lengths` you'd like to test is approximate, since the locally-available tokenizer won't exactly match the one used internally by the model
- The set of `output_lengths` you'd like to test may not always be reached, if the model stops generating early for the given prompts.
- The `requests_per_combination` impacts both the time to run the test and the quality of your output statistics. Note for example that it doesn't make sense to consider p95 or p99 latency on a dataset with only 10 requests!
- A higher number of concurrent `clients` will speed up the overall test run, but could cause problems if you reach quota limits (on as-a-service models) or high request volumes that start to impact response latency (see the "Load testing" section below for more details!)

Similar to low-level test Runners, the `output_path` can be used to configure where the test result data should be saved (either locally or on the Cloud).

In [ ]:
from llmeter.experiments import LatencyHeatmap

latency_heatmap = LatencyHeatmap(
    endpoint=bedrock_endpoint_stream,
    clients=10,
    requests_per_combination=50,
    output_path=f"outputs/{bedrock_endpoint_stream.model_id}/heatmap",
    source_file="MaryShelleyFrankenstein.txt",
    input_lengths=[50, 500, 1000],
    output_lengths=[128, 512, 1024],
    create_payload_fn=prompt_fn,
)

heatmap_results = await latency_heatmap.run()

print(heatmap_results)

(Assuming you installed LLMeter with the `plotting` extras, as shown at the start of this notebook), you'll be able to plot the heatmap results visually to explore how the latency varies with input and output token count:

In [ ]:
latency_heatmap.plot_heatmap()

For many models the overall `time_to_last_token` depends more strongly on the number of tokens *generated* by the model (`num_tokens_output`), while the `time_to_first_token` depends more strongly on the *input* length (`num_tokens_input`) if any significant correlation is present.

## A note on load testing

If you're deploying an LLM on a service like Amazon SageMaker where you **size the underlying infrastructure** (number of compute instances, GPUs, etc) - then of course it will be important to understand how the latency of your deployed model server(s) varies depending on the number of concurrent users / requests.

However, if you're using a large-scale, as-a-service LLM provider like [Amazon Bedrock](https://aws.amazon.com/bedrock/) - it's likely that the request volume for your use-case is insignificant compared to what the service handles overall. In this context, rather than running actual load/volume tests it's more relevant to **check your [API quotas](https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html)** are sufficient for your planned workload - and request increases if necessary.

LLMeter does provide a `LoadTest` experiment to help measure how latency and throughput change as a function of the number of concurrent clients / requests, but for the above reasons we won't cover it in this example. Check the [SageMaker JumpStart example notebook](./LLMeter%20with%20Amazon%20SageMaker%20JumpStart.ipynb) for example code, which should work just fine with Bedrock endpoints too.